Шаг 1. Подготовка входных данных

In [3]:
import torch

x = [
  [1, 0, 1, 0], # Input 1
  [0, 2, 0, 2], # Input 2
  [1, 1, 1, 1]  # Input 3
 ]
x = torch.tensor(x, dtype=torch.float32)

Шаг 2. Инициализация весов

In [4]:

w_key = [
  [0, 0, 1],
  [1, 1, 0],
  [0, 1, 0],
  [1, 1, 0]
]
w_query = [
  [1, 0, 1],
  [1, 0, 0],
  [0, 0, 1],
  [0, 1, 1]
]
w_value = [
  [0, 2, 0],
  [0, 3, 0],
  [1, 0, 3],
  [1, 1, 0]
]
w_key = torch.tensor(w_key, dtype=torch.float32)
w_query = torch.tensor(w_query, dtype=torch.float32)
w_value = torch.tensor(w_value, dtype=torch.float32)

Шаг 3. Получение ключа, запроса и значения

In [5]:

keys = x @ w_key
querys = x @ w_query
values = x @ w_value

print(keys)

print(querys)

print(values)

tensor([[0., 1., 1.],
        [4., 4., 0.],
        [2., 3., 1.]])
tensor([[1., 0., 2.],
        [2., 2., 2.],
        [2., 1., 3.]])
tensor([[1., 2., 3.],
        [2., 8., 0.],
        [2., 6., 3.]])


Шаг 4. Рассчёт показателей внимания

In [6]:
attn_scores = querys @ keys.T

print(attn_scores)

tensor([[ 2.,  4.,  4.],
        [ 4., 16., 12.],
        [ 4., 12., 10.]])


Шаг 5. Рассчёт softmax(scores - баллы)

In [11]:
from torch.nn.functional import softmax

attn_scores_softmax = softmax(attn_scores, dim=-1)

attn_scores_softmax = [
  [0.0, 0.5, 0.5],
  [0.0, 1.0, 0.0],
  [0.0, 0.9, 0.1]
]
attn_scores_softmax = torch.tensor(attn_scores_softmax)

print(attn_scores_softmax)

tensor([[0.0000, 0.5000, 0.5000],
        [0.0000, 1.0000, 0.0000],
        [0.0000, 0.9000, 0.1000]])


Шаг 6. Умножаем баллы на значения

In [8]:

weighted_values = values[:,None] * attn_scores_softmax.T[:,:,None]

print(weighted_values)

tensor([[[0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000]],

        [[1.0000, 4.0000, 0.0000],
         [2.0000, 8.0000, 0.0000],
         [1.8000, 7.2000, 0.0000]],

        [[1.0000, 3.0000, 1.5000],
         [0.0000, 0.0000, 0.0000],
         [0.2000, 0.6000, 0.3000]]])


Шаг 7. Суммируем взвешенные значения

In [9]:
outputs = weighted_values.sum(dim=0)

print(outputs)

tensor([[2.0000, 7.0000, 1.5000],
        [2.0000, 8.0000, 0.0000],
        [2.0000, 7.8000, 0.3000]])
